In [1]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
from model import create_model
import datetime
from config_file import config
from keras import callbacks
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import GPUOptions,Session
from tensorflow.compat.v1 import InteractiveSession

importing Jupyter notebook from model.ipynb


Using TensorFlow backend.


importing Jupyter notebook from config_file.ipynb


In [2]:
# Set tensorboard storage directory
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Create a directory
os.makedirs(log_dir)
Batch_size = config["model"]["BATCH_SIZE"]
FEATURES_SIZE = config["model"]["FEATURES_SIZE"]

In [3]:
Batch_size

64

In [4]:
FEATURES_SIZE

9

## TensorBoardcallback()Function comments
 keras API  https://keras.io/callbacks/#tensorboard
 TensorBoardcallback=tf.keras.callbacks.TensorBoard(log_dir='./logs', 
                  histogram_freq=0, write_graph=True, write_images=False, 
                  embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
 ### The log_dir and histogram parameters are defined in this model, that is, the histogram is calculated, but the specifics have not been understood
 
 ## callbacks.ModelCheckpoint()Function comments
 keras API-callbacks  https://keras.io/callbacks/
   keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

### In this model, val_accuracy is used as the criterion for determining the best performance model, that is, "monitor = 'val_accuracy" ". save_best_only = True Saves the best performing model on the validation set.

## callbacks.EarlyStopping()Function comments

keras API-earlystopping  https://keras.io/callbacks/#earlystopping
keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', 
                   baseline=None, restore_best_weights=False)
### Set patience = 2 in this model, that is, after two epochs, val_accuracy does not increase, that is, to terminate training and reduce overfitting

## model.fit()Function comments
keras API -model.fit()https://keras.io/models/model/#fit
fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0,          validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)

### In this model, 0.2 data is extracted from the original training set as the validation set to determine the state of the model, and training is stopped when appropriate. The validation set cannot change the hyperparameters of the model.


   

In [ ]:
def train_model(features_train,lables_train):
    # Specify the use of a specific GPU, you do not need to specify
    os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
    
    model = create_model()
    
    # print(model.get_weights()) Output the initial parameters of the model
    
    # See Test_model.ipynb for comments
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    # Creating Keras callbacks 
    
    tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    filepath = "training_checkpoints/weights.best.hdf5"
    #filepath =  'training_checkpoints/weights.{epoch:02d}-{val_accuracy:.4f}.hd'
    model_checkpoint_callback = callbacks.ModelCheckpoint(
       filepath, period=1,monitor='val_accuracy',verbose=1, 
                            save_best_only=True)
    os.makedirs('training_checkpoints/', exist_ok=True)
    early_stopping_checkpoint = callbacks.EarlyStopping(patience=2)

    history = model.fit(
              features_train,
              lables_train,
              batch_size = Batch_size,
              epochs=10, 
              #shuffle = True,
              validation_split = 0.2,
              callbacks=[tensorboard_callback,
                         model_checkpoint_callback,
                         early_stopping_checkpoint]
    )
    
    
    return history